##### **EPOCH** = 1 forward and backward pass of all training samples
##### **BATCH SIZE** = no. of training samples in 1 forward and backward pass 
##### **NO. OF ITERATIONS** = no. of passes, each pass using [batch size] no. of samples
 > *Ex. 100 samples , batch size = 20---> 100/20 = 5 iteration for 1 epoch*

In [2]:
import os
os.chdir('/content/')

In [3]:
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
import numpy as np
import math

### **Dataset**

In [4]:
class WineDataset(Dataset):

  def __init__(self):
    # data loading
    xy = np.loadtxt('wine.csv',delimiter=',',dtype=np.float32,skiprows=1)
    self.x = torch.from_numpy(xy[:,1:])
    self.y = torch.from_numpy(xy[:,[0]])
    self.n_samples = xy.shape[0]

  def __getitem__(self,index):
   return self.x[index],self.y[index]

  def __len__(self):
    return self.n_samples

dataset = WineDataset()
# just print
first_data = dataset[0]
features,labels = first_data
print(features,labels)

tensor([1.4230e+01, 1.7100e+00, 2.4300e+00, 1.5600e+01, 1.2700e+02, 2.8000e+00,
        3.0600e+00, 2.8000e-01, 2.2900e+00, 5.6400e+00, 1.0400e+00, 3.9200e+00,
        1.0650e+03]) tensor([1.])


### **DataLoader**

To perform all operations (optimization) on batchwise loaded data (if dataset is too large)

In [11]:
dataloader = DataLoader(dataset=dataset,batch_size=4,shuffle = True,num_workers=2)
dataiter = iter(dataloader)
data = dataiter.next()
features,labels = data
print(features,labels)

# batch size = 4 ; we get 4 features and corrusponding 4 targets.

tensor([[1.3640e+01, 3.1000e+00, 2.5600e+00, 1.5200e+01, 1.1600e+02, 2.7000e+00,
         3.0300e+00, 1.7000e-01, 1.6600e+00, 5.1000e+00, 9.6000e-01, 3.3600e+00,
         8.4500e+02],
        [1.3860e+01, 1.3500e+00, 2.2700e+00, 1.6000e+01, 9.8000e+01, 2.9800e+00,
         3.1500e+00, 2.2000e-01, 1.8500e+00, 7.2200e+00, 1.0100e+00, 3.5500e+00,
         1.0450e+03],
        [1.3050e+01, 2.0500e+00, 3.2200e+00, 2.5000e+01, 1.2400e+02, 2.6300e+00,
         2.6800e+00, 4.7000e-01, 1.9200e+00, 3.5800e+00, 1.1300e+00, 3.2000e+00,
         8.3000e+02],
        [1.3170e+01, 2.5900e+00, 2.3700e+00, 2.0000e+01, 1.2000e+02, 1.6500e+00,
         6.8000e-01, 5.3000e-01, 1.4600e+00, 9.3000e+00, 6.0000e-01, 1.6200e+00,
         8.4000e+02]]) tensor([[1.],
        [1.],
        [1.],
        [3.]])


In [12]:
# training loop
epochs = 2
total_samples = len(dataset)
n_iterations = math.ceil(total_samples/4)
print(total_samples,n_iterations)

178 45


In [14]:
for epoch in range(epochs):
  for i, (inputs,labels) in enumerate(dataloader):
    # fwd,bwd,update
    if (i+1) % 5 ==0:
      print("epoch {0}/{1},step {2}/{3}, inputs {4}".format(epoch+1,epochs,
                                                i+1,n_iterations,inputs.shape))

epoch 1/2,step 5/45, inputs torch.Size([4, 13])
epoch 1/2,step 10/45, inputs torch.Size([4, 13])
epoch 1/2,step 15/45, inputs torch.Size([4, 13])
epoch 1/2,step 20/45, inputs torch.Size([4, 13])
epoch 1/2,step 25/45, inputs torch.Size([4, 13])
epoch 1/2,step 30/45, inputs torch.Size([4, 13])
epoch 1/2,step 35/45, inputs torch.Size([4, 13])
epoch 1/2,step 40/45, inputs torch.Size([4, 13])
epoch 1/2,step 45/45, inputs torch.Size([2, 13])
epoch 2/2,step 5/45, inputs torch.Size([4, 13])
epoch 2/2,step 10/45, inputs torch.Size([4, 13])
epoch 2/2,step 15/45, inputs torch.Size([4, 13])
epoch 2/2,step 20/45, inputs torch.Size([4, 13])
epoch 2/2,step 25/45, inputs torch.Size([4, 13])
epoch 2/2,step 30/45, inputs torch.Size([4, 13])
epoch 2/2,step 35/45, inputs torch.Size([4, 13])
epoch 2/2,step 40/45, inputs torch.Size([4, 13])
epoch 2/2,step 45/45, inputs torch.Size([2, 13])


### **Dataset Transforms**

> ***torchvision.datasets*** provides MNIST, CIFER, COCO, Fashion-mnist etc.

**Transforms** can be applied to PIL images, tensors, ndarrays, or custom data
during creation of the DataSet.

complete list of built-in transforms: 
https://pytorch.org/docs/stable/torchvision/transforms.html

On Images
---------
CenterCrop, Grayscale, Pad, RandomAffine,
RandomCrop, RandomHorizontalFlip, RandomRotation
Resize, Scale

On Tensors
----------
LinearTransformation, Normalize, RandomErasing

Conversion
----------
ToPILImage: from tensor or ndrarray

ToTensor : from numpy.ndarray or PILImage

Generic
-------
Use Lambda 

Custom
------
Write own class

Compose multiple Transforms
---------------------------
composed = transforms.Compose([Rescale(256),
                               RandomCrop(224)])

In [15]:
'''
import torch
import torchvision

dataset = torchvision.datasets.MNIST(
  root = './data',transform = torchvision.transforms.ToTensor()
)
'''

"\nimport torch\nimport torchvision\n\ndataset = torchvision.datasets.MNIST(\n  root = './data',transform = torchvision.transforms.ToTensor()\n)\n"

In [22]:
# write our own transform and apply it our own dataset

class WineDataset(Dataset):

  def __init__(self,transform = None):
    # data loading
    xy = np.loadtxt('wine.csv',delimiter=',',dtype=np.float32,skiprows=1)
    self.x = xy[:,1:]
    self.y = xy[:,[0]]
    self.n_samples = xy.shape[0]

    self.transform = transform

  def __getitem__(self,index):
   sample = self.x[index],self.y[index]

   if self.transform:
     sample = self.transform(sample)

   return sample

  def __len__(self):
    return self.n_samples

class ToTensor:
  def __call__(self,sample):
    inputs,targets = sample
    return torch.from_numpy(inputs),torch.from_numpy(targets)

class MulTransform:
  def __init__(self,factor):
    self.factor = factor

  def __call__(self,sample):
    inputs,targets = sample
    inputs *= self.factor
    return inputs,targets

dataset = WineDataset(transform = ToTensor())
# just print
first_data = dataset[0]
features,labels = first_data
print(type(features),type(labels))
print('*'*10)


dataset = WineDataset(transform = None)
# just print
first_data = dataset[0]
features,labels = first_data
print(features)
print(type(features),type(labels))
print('*'*10)


# using Compose transform (composition of 2 transform)
composed = torchvision.transforms.Compose([ToTensor(),MulTransform(4)])
dataset = WineDataset(transform = composed)
# just print
first_data = dataset[0]
features,labels = first_data
print(features)
print(type(features),type(labels))
print('*'*10)

<class 'torch.Tensor'> <class 'torch.Tensor'>
**********
[1.423e+01 1.710e+00 2.430e+00 1.560e+01 1.270e+02 2.800e+00 3.060e+00
 2.800e-01 2.290e+00 5.640e+00 1.040e+00 3.920e+00 1.065e+03]
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
**********
tensor([5.6920e+01, 6.8400e+00, 9.7200e+00, 6.2400e+01, 5.0800e+02, 1.1200e+01,
        1.2240e+01, 1.1200e+00, 9.1600e+00, 2.2560e+01, 4.1600e+00, 1.5680e+01,
        4.2600e+03])
<class 'torch.Tensor'> <class 'torch.Tensor'>
**********
